# Pop2Piano - Smart Inference 🎹🎵

Generate piano covers with **automatic Arabic music detection** and **smart composer selection**.

## Features:
- **Auto-detect** if a song is Arabic and which Maqam it uses
- **Auto-select** the best composer (Arr. Style) from 21 options
- **Maqam post-processing** to quantize notes to Arabic scales
- Works with both Arabic and Western songs

## Steps:
1. Install Dependencies
2. Clone Repository
3. Load Model
4. **Analyze Audio** (auto-detect maqam & Arabic features)
5. **Smart Generate** (try multiple composers, pick best)
6. Download MIDI

In [ ]:
# @title 1. Install Dependencies 📦
import os
import sys
import subprocess

print("Installing dependencies... ⏳")

# Install Python packages (stable versions only)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "transformers", "essentia", "librosa", "resampy", "pretty_midi", "midi2audio", "scipy", "yt-dlp", "imageio-ffmpeg", "omegaconf", "joblib", "tqdm"])

# Install System packages
os.system("sudo apt-get update -q")
os.system("sudo apt-get install -y -q fluidsynth fluid-soundfont-gm ffmpeg")

print("Done! ✅")

In [ ]:
# @title 2. Apply Magic Fixes (DISABLED) 🪄
# We DISABLED the mocking because Inference requires real Essentia to analyze audio rhythm.
# Only enable this if you are debugging environment issues and NOT running inference.

print("Skipping Mocking... We need the real Essentia library! 🎵")

# import sys
# from unittest.mock import MagicMock
# from importlib.machinery import ModuleSpec

# # Fix for missing essentia
# mock_essentia = MagicMock()
# # Important fix for Python 3.12+ (spec must be set)
# mock_essentia.__spec__ = ModuleSpec(name='essentia', loader=None)

# sys.modules["essentia"] = mock_essentia
# sys.modules["essentia.standard"] = MagicMock()

# print("Essentia mocked successfully! 🧙‍♂️")

In [ ]:
# @title 3. Clone Repository 🐙
import os

repo_url = "https://github.com/kareemkamal10/pop2piano.git"
repo_name = "pop2piano"

# Detect environment
IN_KAGGLE = os.path.exists('/kaggle')
work_dir = "/kaggle/working" if IN_KAGGLE else "/content"

%cd {work_dir}

if os.path.exists(repo_name):
    print("Repository already cloned. Pulling latest changes...")
    %cd {repo_name}
    !git pull
else:
    print(f"Cloning {repo_url}...")
    !git clone {repo_url}
    %cd {repo_name}

print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title 4. Load Model 🎹
from transformers import Pop2PianoForConditionalGeneration, Pop2PianoProcessor
import torch
import os

# Detect environment
IN_COLAB = 'google.colab' in str(get_ipython())
IN_KAGGLE = os.path.exists('/kaggle')

# Load Model
print("Loading Model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Pop2PianoForConditionalGeneration.from_pretrained("sweetcocoa/pop2piano").to(device)
processor = Pop2PianoProcessor.from_pretrained("sweetcocoa/pop2piano")
print(f"Model Loaded on {device}!")

In [ ]:
# @title 5. Upload Audio File 📤
import librosa

# Get Audio File
if IN_COLAB:
    from google.colab import files
    print("Please upload your audio file (MP3/WAV)...")
    uploaded = files.upload()
    audio_filename = next(iter(uploaded))
elif IN_KAGGLE:
    import glob
    audio_files = (
        glob.glob("/kaggle/input/**/*.mp3", recursive=True)
        + glob.glob("/kaggle/input/**/*.wav", recursive=True)
    )
    if audio_files:
        audio_filename = audio_files[0]
        print(f"Found audio: {audio_filename}")
    else:
        raise FileNotFoundError("No audio file found! Add an MP3/WAV via 'Add Data'.")
else:
    audio_filename = "your_audio.mp3"

print(f"Processing: {audio_filename}")
audio, sr = librosa.load(audio_filename, sr=44100)
print(f"Duration: {len(audio)/sr:.1f}s, Sample rate: {sr}Hz")

In [ ]:
# @title 6. Analyze Audio (Auto-Detect Maqam & Arabic Features) 🔍
from smart_inference import analyze_audio, print_analysis_report

analysis = analyze_audio(audio, sr)
print_analysis_report(analysis)

In [ ]:
# @title 7. Smart Generate (Auto-Select Best Composer + Maqam) 🎹✨
from smart_inference import SmartPop2Piano

smart = SmartPop2Piano(model, processor, device=device)

# ╔══════════════════════════════════════════════════════════════╗
# ║  SETTINGS - Change these to control generation              ║
# ╠══════════════════════════════════════════════════════════════╣
# ║                                                              ║
# ║  auto_detect_maqam: True = auto-detect Arabic maqam         ║
# ║  maqam: None = auto, or force: "hijaz", "bayyati", "rast",  ║
# ║         "saba", "nahawand", "kurd", "ajam", "nikriz"        ║
# ║  composer: None = auto-select best, or force "composer1"-21  ║
# ║  auto_select_composer: True = try 5 composers, pick best    ║
# ║                                                              ║
# ╚══════════════════════════════════════════════════════════════╝

result = smart.generate(
    audio=audio,
    sr=sr,
    # --- Auto-Detection ---
    auto_detect_maqam=True,       # Auto-detect maqam from audio
    maqam=None,                   # None=auto, or force: "hijaz", "bayyati", etc.
    # --- Composer Selection ---
    composer=None,                # None=auto, or force: "composer1", "composer2", etc.
    auto_select_composer=True,    # Try multiple composers, pick the best
    # --- Post-Processing ---
    apply_post_processing=True,   # Apply maqam + piano rules
    simplify_chords=True,         # Reduce complex chords
    quantize_rhythm=True,         # Snap to beat grid
    humanize=False,               # Add human-like velocity variation
)

print(f"\nBest composer: {result.best_composer}")
print(f"Quality score: {result.best_score:.2f}")
if result.applied_maqam:
    print(f"Applied maqam: {result.applied_maqam}")

In [ ]:
# @title 8. Save & Download MIDI 💾
import os

output_midi_name = "output_midi.mid"
if IN_KAGGLE:
    output_midi_name = "/kaggle/working/output_midi.mid"

result.best_midi.write(output_midi_name)
print(f"Saved to {output_midi_name}")

# Also save results from all composers for comparison
output_dir = "all_composers"
os.makedirs(output_dir, exist_ok=True)
base_name = os.path.splitext(os.path.basename(audio_filename))[0]

for cr in sorted(result.all_results, key=lambda r: r.score, reverse=True):
    path = os.path.join(output_dir, f"{base_name}_{cr.composer}_score{cr.score:.2f}.mid")
    cr.midi.write(path)

print(f"\nAll {len(result.all_results)} composer results saved to '{output_dir}/' folder.")
print("Compare them to find your favorite!")

# Show ranking
print("\nComposer Ranking:")
for i, cr in enumerate(sorted(result.all_results, key=lambda r: r.score, reverse=True), 1):
    marker = " <-- BEST" if cr.composer == result.best_composer else ""
    print(f"  {i}. {cr.composer}: score={cr.score:.2f}, notes={cr.note_count}{marker}")

# Download (Colab only)
if IN_COLAB:
    from google.colab import files
    files.download(output_midi_name)
    print("\nDownloading best MIDI...")

## Optional: Manual Override

If auto-detection didn't get it right, use the cell below to force specific settings.

**Available Maqamat:** `rast`, `bayyati`, `hijaz`, `saba`, `nahawand`, `kurd`, `ajam`, `sikah`, `nikriz`, `hijaz_kar`, `nawa_athar`, `athar_kurd`

**Composers:** `composer1` through `composer21`

In [ ]:
# @title 9. (Optional) Re-Generate with Manual Settings 🎛️

# Change these to your preferred settings:
FORCE_MAQAM = "hijaz"       # Set to None for auto, or: "hijaz", "bayyati", "rast", etc.
FORCE_COMPOSER = None        # Set to None for auto, or: "composer1", "composer5", etc.
TRY_ALL_21 = False           # Set True to try ALL 21 composers (slower but thorough)

composers = [f"composer{i}" for i in range(1, 22)] if TRY_ALL_21 else None

manual_result = smart.generate(
    audio=audio,
    sr=sr,
    auto_detect_maqam=False,
    maqam=FORCE_MAQAM,
    composer=FORCE_COMPOSER,
    auto_select_composer=(FORCE_COMPOSER is None),
    composers_to_try=composers,
    apply_post_processing=True,
)

output_name = f"output_{FORCE_MAQAM or 'auto'}_{manual_result.best_composer}.mid"
manual_result.best_midi.write(output_name)
print(f"\nSaved: {output_name}")

if IN_COLAB:
    from google.colab import files
    files.download(output_name)